In [1]:
import importlib
from osgeo import gdal
import pprint
import json
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import Counter

# 0 => clear land pixel
# 1 => clear water pixel
# 2 => cloud shadow
# 3 => snow
# 4 => cloud

In [15]:
import os

directory = f"/Volumes/X/Data/fusion-s1-s2/s2/fmask/"  # replace with the path to your directory
fmask_paths = [
    f"/Volumes/X/Data/fusion-s1-s2/s2/fmask/" +
    folder_name +
    "/" +
    folder_name +
    "_fmask.tif"
    for folder_name in os.listdir(directory)
]

In [3]:
MAP_CLOUD = {
    0: "clear_land",
    1: "cloud_water", 
    2: "cloud_shadow",
    3: "snow",
    4: "cloud",
    255: "no_observation"
}

In [17]:
import os
from osgeo import gdal

def fmask_double(input_path, output_path):
    # Open raster and get band
    in_ds = gdal.Open(input_path)
    in_band = in_ds.GetRasterBand(1)

    # Multiply output size by 3 
    out_rows = in_band.YSize * 2
    out_columns = in_band.XSize * 2

    # Create new data source (raster)
    gtiff_driver = gdal.GetDriverByName('GTiff')
    out_ds = gtiff_driver.Create(output_path, out_columns, out_rows)
    out_ds.SetProjection(in_ds.GetProjection())
    geotransform = list(in_ds.GetGeoTransform())

    # Edit the geotransform so pixels are one-sixth previous size
    geotransform[1] /= 2
    geotransform[5] /= 2
    out_ds.SetGeoTransform(geotransform)

    data = in_band.ReadAsArray(buf_xsize=out_columns, buf_ysize=out_rows)  # Specify a larger buffer size when reading data
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(data)

    out_band.FlushCache()
    out_band.ComputeStatistics(False)
    out_ds.BuildOverviews('average', [2, 4, 8, 16, 32, 64])

    del out_ds

In [20]:
for fmask_path in fmask_paths:
    output_path = fmask_path.split(".")[0] + "_d.tif"
    fmask_double(fmask_path, output_path)

In [5]:
json_data = {}

FMASK_DIRECTORY = f"/Volumes/X/Data/fusion-s1-s2/s2/fmask/"

for fmask_path in os.listdir(FMASK_DIRECTORY):
    path_to_crops = f"{FMASK_DIRECTORY}{fmask_path}/{fmask_path}_crops/"
    for im_path in os.listdir(path_to_crops):
        x, y = int(im_path.split("_")[-1].split(".")[0]), int(im_path.split("_")[-2])
        gdal_data = gdal.Open(im_path)
        data = gdal_data.ReadAsArray()
        
        shape = data.size
        
        count = dict(Counter(itertools.chain(*data)))
        map = {MAP_CLOUD[k]: round((v.shape)*100, 2) for k, v in count.items()}
        
        json_data[im_path.split("/")[-1]] = dict(sorted(map.items(), key=lambda x: -x[1]))

with open("fmask_stats.json", "w") as f:
    json.dump(json_data, f)

(5490, 5490)
(5490, 5490)
(5490, 5490)
(5490, 5490)


KeyboardInterrupt: 